In [211]:
def test_proc(path,file):
    import pandas as pd
    import numpy as np
    # Читаем файл
    train=pd.read_csv(path+file,delimiter='\t',encoding='utf-8')
    name=file.split('.')[0]
    
    # Разбиваем признаки
    spl_train=train.features.str.split(',',expand=True) 
    types=spl_train[0].unique()
    features=['feature_{}_{}'.format(types[0],i) for i in range(1,257)]
    columns_name=['type_of_features']+features 
    train[columns_name] = spl_train
    
    # Создаем окончательную таблицу 
    train_proc=pd.DataFrame()
    train_proc['id_job']=train['id_job']
    
    # Проводим стандартизацию
    for i in range(len(features)):
        train[features[i]]=pd.to_numeric(train[features[i]])
        train_proc['feature_{}_stand_{}'.format(types[0],i+1)]=(train[features[i]]-np.mean(train[features[i]]))/np.std(train[features[i]])
    # Находим индекс максимального значения признака, максимальное значение 
    # и признак в котором это макс значение(нужно для нахождения абсолютного отклонения ) для вакансии 
    train_proc['max_feature_{}_index'.format(types[0])]=pd.to_numeric(train[features].idxmax(axis=1).str.split('_',expand=True)[2])
    train_proc['max_feature_{}'.format(types[0])]=train[features].idxmax(axis=1)
    train_proc['max_feature_{}_int'.format(types[0])]=pd.to_numeric(train[features].max(axis=1))
    
    df=pd.DataFrame()
    df['features']=features
    df['mean_feature']=0.0
    for i in range(len(features)):
        df['mean_feature'][i]=np.mean(train[features[i]])
    
    train_proc=train_proc.merge(df,left_on='max_feature_{}'.format(types[0]),right_on='features')
    #Находим абсолютное отклонение
    train_proc['max_feature_{}_abs_mean_diff'.format(types[0])]=abs(train_proc['max_feature_{}_int'.format(types[0])]-train_proc['mean_feature'])
    #Дропаем промежуточные колонки
    train_proc=train_proc.drop(['features','max_feature_{}'.format(types[0]),'max_feature_{}_int'.format(types[0]),'mean_feature'],axis=1)
    #Запишем  окончательный файл 
    train_proc.to_csv('{}_proc.tsv'.format(name))

    
test_proc('C:\\Users\\andre\\Documents\\','test.tsv')